In [1]:
import numpy as np
#import lightfm
from lightfm.datasets import fetch_movielens

movielens = fetch_movielens()

ModuleNotFoundError: No module named 'lightfm'

In [20]:
train = movielens['train']
test = movielens['test']
print (type(train))

<class 'scipy.sparse.coo.coo_matrix'>


In [4]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM(learning_rate=0.05, loss='bpr')
model.fit(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.59, test 0.10.
AUC: train 0.89, test 0.86.


In [24]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM(learning_rate=0.05, loss='warp')

model.fit_partial(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.60, test 0.11.
AUC: train 0.93, test 0.90.


In [25]:
#print(test)
user = np.array([1])
movie = np.array([10])
a = model.predict(user,movie)
print(a)

[-1.65489483]


# copy-paste from Raval

In [49]:
# copy-paste from Raval
from lightfm import LightFM
import pandas as pd
import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score


movielens = fetch_movielens()

data = fetch_movielens(min_rating=4.)
train = data['train']
test = data['test']
model = LightFM(learning_rate=0.05, loss='warp')
model.fit(data['train'],epochs=20,num_threads=2)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))



def recommendation(model, data, user_ids):
    n_users, n_items = data['train'].shape
    
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        print(data['item_labels'])
        print(data['train'].shape)
        scores = model.predict(user_id, np.asarray(range(n_items)))
        top_items = data['item_labels'][np.argsort(-scores)]
        
        print("User:",user_id)
        print("Rated:")
        for i in known_positives[:5]:
            print("      ",i)
        print("Recommended:")
        for i in top_items[:5]:
            print("      ",i)
    return


recommendation(model, data, [1,2,3])

Precision: train 0.52, test 0.08.
AUC: train 0.95, test 0.92.
['Toy Story (1995)' 'GoldenEye (1995)' 'Four Rooms (1995)' ...
 'Sliding Doors (1998)' 'You So Crazy (1994)'
 'Scream of Stone (Schrei aus Stein) (1991)']
(943, 1682)
User: 1
Rated:
       Toy Story (1995)
       Postino, Il (1994)
       Birdcage, The (1996)
       Fargo (1996)
       Truth About Cats & Dogs, The (1996)
Recommended:
       English Patient, The (1996)
       L.A. Confidential (1997)
       Titanic (1997)
       Contact (1997)
       Air Force One (1997)
['Toy Story (1995)' 'GoldenEye (1995)' 'Four Rooms (1995)' ...
 'Sliding Doors (1998)' 'You So Crazy (1994)'
 'Scream of Stone (Schrei aus Stein) (1991)']
(943, 1682)
User: 2
Rated:
       Return of the Jedi (1983)
       Event Horizon (1997)
       Schindler's List (1993)
       Paradise Lost: The Child Murders at Robin Hood Hills (1996)
       Mother (1996)
Recommended:
       L.A. Confidential (1997)
       Contact (1997)
       Scream (1996)
       Titani

# Custom dataset loading

In [42]:
def load_data():
    import numpy as np
    import pandas as pd
    from lightfm.datasets import fetch_movielens
    from lightfm.data import Dataset

    n_ratings = 500000

    # Load the file into a dataframe
    #table = pd.read_table("~/lightfm_data/movielens100k/ml-100k/u.data",names=['userId','movieId','rating','timestamp'])
    table = pd.read_csv("ratings.csv")
    #### TESTING ITEMS LABELING
    file = pd.read_csv("movies.csv")
    file['title'].str.split()
    user1movies = []
    for i in range(len(table['userId'])):
        if table['userId'][i] == 2:
            break
        else:
            user = file[file['movieId']==table['movieId'][i]]
            #print(user['title'].values[0].strip())
            user1movies.append(user['title'].values[0].strip())
#            movie = user['title'].split(',')[0]
    #### END ITEMS LABELING TEST
    print(len((table[ table['userId']==1 ])['userId']))##################################################################################    
    table = table[['userId','movieId']].iloc[:n_ratings]
    table = table.astype(np.int32)    

    # Perform the mapping between users and movies needed for creating interaction matrices
    data = Dataset()
    data.fit(table['userId'], table['movieId'])

    # Get some useful numbers
    num_users, num_items = data.interactions_shape()
    print('Num users: {}, num_items {}.'.format(num_users, num_items))

    # Creating interaction matrices
    tuples = [tuple(x) for x in table[['userId','movieId']].values]
    (interactions, weights) = data.build_interactions(tuples)

    return interactions,user1movies


#i, u1m= load_data()

def label_items():
    import pandas as pd
    import numpy as np
    
    file = pd.read_csv("movies.csv")
    #file['title'] = file['title'].apply(lambda x: x.split(',')[0])
    file = np.asarray(file['title'].tolist())
    return file


def get_metrics(interactions,u1m):
    from lightfm.cross_validation import random_train_test_split
    from lightfm import LightFM
    from lightfm.evaluation import precision_at_k
    from lightfm.evaluation import auc_score

    (train_set, test_set) = random_train_test_split(interactions, test_percentage=0.2, random_state=None)
    ##########  TESTING ITEMS LABELING
    movie_labels = label_items()
    print(movie_labels)
    return
    user = 0
    #print(movie_labels[train_set.tocsr()[user].indices])
    #labels = ''.join(list(movie_labels[train_set.tocsr()[user].indices]))
    labels = movie_labels[train_set.tocsr()[user].indices]
    #strlist = ''.join(labels)
    
    print(len(u1m))
    print(len(labels))
    print(labels)
#    for thing in u1m:
#        if not thing in labels:
#            print("BULLSHIT",thing)
#            #return
#    print("CLEAN!")
    return
    ##########  ENDING ITEMS LABELING TEST
    model = LightFM(loss='warp')
    model.fit(train_set, epochs=20, num_threads=2)

    train_precision = precision_at_k(model, train_set, k=20).mean()
    test_precision = precision_at_k(model, test_set, k=20).mean()

    train_auc = auc_score(model, train_set).mean()
    test_auc = auc_score(model, test_set).mean()


    print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
    print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))
    return
get_metrics(i,u1m)

def recommendation(model, data, user_ids):
    n_users, n_items = data['train'].shape
    
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        # hay que construir otem_labels como en el fetch_movielens
        # hecho en la func "label_items"
        
        
        scores = model.predict(user_id, np.asarray(range(n_items)))
        top_items = data['item_labels'][np.argsort(-scores)]
        
        print("User:",user_id)
        print("Rated:")
        for i in known_positives[:5]:
            print("      ",i)
        print("Recommended:")
        for i in top_items[:5]:
            print("      ",i)
    return


175
Num users: 3374, num_items 12277.
['Toy Story (1995)' 'Jumanji (1995)' 'Grumpier Old Men (1995)' ...
 'The Pirates (2014)' 'Rentun Ruusu (2001)' 'Innocence (2014)']


In [84]:
ints = load_data()
get_metrics(ints)

Num users: 3374, num_items 12277.
Precision: train 0.43, test 0.10.
AUC: train 0.97, test 0.95.


In [43]:
import pandas as pd

df1 = pd.read_csv("ratings.csv")
df2 = pd.read_csv("movies.csv")

In [45]:
df3 = df1.join(df2,on='movieId')

ValueError: columns overlap but no suffix specified: Index(['movieId'], dtype='object')